When we define a new type it's called a class. The convention in Python is that classes start with capital letter.

Define a new type, encapsulate data inside of that type, define a bunch of methods that define the behaviors that you can perform on the objects of that type.

You'll notice that there are two interesting looking method names here, init and str, and these methods look just like functions. The only reason that we don't call them functions is that they're defined inside the context of a class. So they can only operate on objects of this type Character, they can't be called directly in any other way.

init is called the initializer.

self variable is reference to the new object.

In [ ]:
#__method__ are never called directly. Python calls them behind your back.

In [ ]:
class Character:
    def __init__(self, name, initial_health):
        self.name = name
        # field is referenced by a dot notation
        self.health = initial_health
        self.inventory = []
        
        # name, health, and inventory are fields inside the object.
        # init object is the initializer, it doesn't return anything. 
        
    def __str__(self):
        s  = "Name: " + self.name
        s += " Health: " + str(self.health)
        s += " Inventory: " + str(self.inventory)
        return s
    
    # The functions below are also called behaviors
    
    def grab(self, item):
        self.inventory.append(item)
        
    def get_health(self):
        return self.health

In [ ]:
def example():
    me = Character("Bob", 20)
    # The class name becomes the constructor. So when I want to create a new character, 
    # I call Character as if it were a function, and then I pass the argument's name and initial health. 
    # You'll see I don't pass self 
    # And that the init function, even though it didn't actually return anything, 
    # returns the new object that was created
    # 
    print str(me)
    # If I use str(me) it will behind my back call __stir__, and it'll pass the object me as self
    
    me.grab("pencil")
    print str(me)
    me.grab("paper")
    print str(me)
    print "Health:", me.get_health()

In [ ]:
example()

** OOP 202 **

In [ ]:
# ball physics code for generic 2D domain
# the functions inside() and normal() encode the shape of the ennvironment

import simplegui
import random
import math

# Canvas size
width = 600
height = 400

# Ball traits
radius = 20
color = "White"

# math helper function
def dot(v, w):
    return v[0] * w[0] + v[1] * w[1]

class RectangularDomain:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.border = 2

    # return if bounding circle is inside the domain    
    def inside(self, center, radius):
        in_width = ((radius + self.border) < center[0] < 
                    (self.width - self.border - radius))
        in_height = ((radius + self.border) < center[1] < 
                     (self.height - self.border - radius))
        return in_width and in_height

    # return a unit normal to the domain boundary point nearest center
    def normal(self, center):
        left_dist = center[0]
        right_dist = self.width - center[0]
        top_dist = center[1]
        bottom_dist = self.height - center[1]
        if left_dist < min(right_dist, top_dist, bottom_dist):
            return (1, 0)
        elif right_dist < min(left_dist, top_dist, bottom_dist):
            return (-1, 0)
        elif top_dist < min(bottom_dist, left_dist, right_dist):
            return (0, 1)
        else:
            return (0, -1)

    # return random location
    def random_pos(self, radius):
        x = random.randrange(radius, self.width - radius - self.border)
        y = random.randrange(radius, self.height - radius - self.border)
        return [x, y]

    # Draw boundary of domain
    def draw(self, canvas):
        canvas.draw_polygon([[0, 0], [self.width, 0], 
                             [self.width, self.height], [0, self.height]],
                             self.border*2, "Red")
        
class CircularDomain:
    def __init__(self, center, radius):
        self.center = center
        self.radius = radius
        self.border = 2
        
    # return if bounding circle is inside the domain    
    def inside(self, center, radius):
        dx = center[0] - self.center[0]
        dy = center[1] - self.center[1]
        dr = math.sqrt(dx ** 2 + dy ** 2)
        return dr < (self.radius - radius - self.border)

    # return a unit normal to the domain boundary point nearest center
    def normal(self, center):
        dx = center[0] - self.center[0]
        dy = center[1] - self.center[1]
        dr = math.sqrt(dx ** 2 + dy ** 2)
        return [dx / dr, dy / dr]
    
    # return random location
    def random_pos(self, radius):
        r = random.random() * (self.radius - radius - self.border)
        theta = random.random() * 2 * math.pi
        x = r * math.cos(theta) + self.center[0]
        y = r * math.sin(theta) + self.center[1]
        return [x, y]
        
    # Draw boundary of domain
    def draw(self, canvas):
        canvas.draw_circle(self.center, self.radius, self.border*2, "Red")
    
class Ball:
    def __init__(self, radius, color, domain):
        self.radius = radius
        self.color = color
        self.domain = domain
        
        self.pos = self.domain.random_pos(self.radius)
        self.vel = [random.random() + .1, random.random() + .1]
        
    # bounce
    def reflect(self):
        norm = self.domain.normal(self.pos)
        norm_length = dot(self.vel, norm)
        self.vel[0] = self.vel[0] - 2 * norm_length * norm[0]
        self.vel[1] = self.vel[1] - 2 * norm_length * norm[1]
    

    # update ball position
    def update(self):
        self.pos[0] += self.vel[0]
        self.pos[1] += self.vel[1]
        if not self.domain.inside(self.pos, self.radius):
            self.reflect()

    # draw
    def draw(self, canvas):
        canvas.draw_circle(self.pos, self.radius, 1, 
                           self.color, self.color)
        

# generic update code for ball physics
def draw(canvas):
    ball.update()
    field.draw(canvas)
    ball.draw(canvas)

field = RectangularDomain(width, height)
# field = CircularDomain([width/2, height/2], 180)
ball = Ball(radius, color, field)
        
frame = simplegui.create_frame("Ball physics", width, height)

frame.set_draw_handler(draw)

frame.start()

In [ ]:
# Particle class example used to simulate diffusion of molecules

import simplegui
import random

# global constants
WIDTH = 600
HEIGHT = 400
PARTICLE_RADIUS = 5
COLOR_LIST = ["Red", "Green", "Blue", "White"]
DIRECTION_LIST = [[1,0], [0, 1], [-1, 0], [0, -1]]


# definition of Particle class
class Particle:
    
    # initializer for particles
    def __init__(self, position, color):
        self.position = position
        self.color = color
        
    # method that updates position of a particle    
    def move(self, offset):
        self.position[0] += offset[0]
        self.position[1] += offset[1]
        
    # draw method for particles
    def draw(self, canvas):
        canvas.draw_circle(self.position, PARTICLE_RADIUS, 1, self.color, self.color)
    
    # string method for particles
    def __str__(self):
        return "Particle with position = " + str(self.position) + " and color = " + self.color


# draw handler
def draw(canvas):
    for p in particle_list:
        p.move(random.choice(DIRECTION_LIST))
    
    for p in particle_list:
        p.draw(canvas)


# create frame and register draw handler
frame = simplegui.create_frame("Particle simulator", WIDTH, HEIGHT)
frame.set_draw_handler(draw)

# create a list of particles
particle_list = []
for i in range(100):
    p = Particle([WIDTH / 2, HEIGHT / 2], random.choice(COLOR_LIST))
    particle_list.append(p)

# start frame
frame.start()

In [ ]:
# Testing template for Particle class


###################################################
# Student should add code for the Particle class here
    


###################################################
# Test code for the Particle class


p = Particle([20, 20], "Red")
print p
print type(p)
p.move([10, 20])
print p
p.move([-15, -25])
print p
print
q = Particle([15, 30], "Green")
print q
print type(q)
q.move([0, 0])
print q


###################################################
# Output from test

#Particle with position = [20, 20] and color = Red
#<class '__main__.Particle'>
#Particle with position = [30, 40] and color = Red
#Particle with position = [15, 15] and color = Red
#
#Particle with position = [15, 30] and color = Green
#<class '__main__.Particle'>
#Particle with position = [15, 30] and color = Green

In [38]:
import turtle

def draw_square():
    window = turtle.Screen()
    window.bgcolor("red")
    
    brad = turtle.Turtle()
    # brad is an instance of class called turtle
    brad.forward(100)
    brad.right(90)
    brad.forward(100)
    brad.right(90)
    brad.forward(100)
    brad.right(90)
    brad.forward(100)
    brad.right(90)
    
    window.exitonclick()
    

In [ ]:
draw_square()

In [2]:
import twilio
print twilio.__version__

6.8.3


In [5]:
def read_text():
    quotes = open('./movie_quotes.txt', 'r')
    # quotes is an object or an instance of File
    # open in turn is calling some __init__ like function to open 
    contents = quotes.read()
    print contents
    quotes.close()

In [4]:
read_text()

-- Houston, we have a problem. (Apollo 13)

-- Mama always said, life is like a box of chocolates. You never know what you are going to get. (Forrest Gump)

-- You cant handle the truth. (A Few Good Men)

-- I believe everything and I believe nothing. (A Shot in the Dark)


In [6]:
import urllib

In [15]:
def check_profanity(text_to_check):
    connection = urllib.urlopen("http://www.wdylike.appspot.com/?q=" + text_to_check)
    output = connection.read()
    connection.close()
    # print output
    if "true" in output:
        print "Profanity alert!"
    elif "flase" in output:
        print "Document has no curse words"
    else:
        print "Could not scan the document properly"

In [16]:
def read_text():
    quotes = open('./movie_quotes.txt', 'r')
    # quotes is an object or an instance of File
    # open in turn is calling some __init__ like function to open 
    contents = quotes.read()
    # print contents
    output = check_profanity(contents)
    print output
    quotes.close()

In [17]:
read_text()

Profanity alert!
None


In [48]:
import webbrowser

class Movie():
    """This class provides a way to store movie related information 
    Second line"""
    
    VALID_RATINGS = ['G', 'PG', 'PG-13', 'R']
    # Valid_ratings is a class variable, available to all instances of a class.
    # Python's Google style guide says that we wanna capitalize the variable names
    # which are not gonna changes (are constant).
    def __init__(self, movie_title, movie_storyline, poster_image, trailer_youtube):
        
        self.title = movie_title
        self.storyline = movie_storyline
        self.poster_image_url = poster_image
        self.trailer_youtube_url = trailer_youtube
        
    def show_trailer(self):
        webbrowser.open(self.trailer_youtube_url)

In [44]:
toy_story = Movie('Toy Story', \
                       'A story of a boy and his toys that come to life', \
                       'https://upload.wikimedia.org/wikipedia/en/1/13/Toy_Story.jpg', \
                       'https://www.youtube.com/watch?v=KYz2wyBy3kc')

In [22]:
print toy_story.storyline

A story of a boy and his toys that come to life


In [29]:
avatar = Movie('Avatar', 'A marine on alien planet', \
               'https://upload.wikimedia.org/wikipedia/en/b/b0/Avatar-Teaser-Poster.jpg', \
              'https://www.youtube.com/watch?v=5PSNL1qE6VY')

In [27]:
print avatar.storyline

AttributeError: Movie instance has no attribute 'storyline'

In [30]:
avatar.show_trailer()

In [31]:
import fresh_tomatoes

In [34]:
movies = [toy_story, avatar]

In [35]:
fresh_tomatoes.open_movies_page(movies)

In [37]:
Movie.VALID_RATINGS

['G', 'PG', 'PG-13', 'R']

In [39]:
turtle.Turtle.__doc__

'RawTurtle auto-creating (scrolled) canvas.\n\n    When a Turtle object is created or a function derived from some\n    Turtle method is called a TurtleScreen object is automatically created.\n    '

In [49]:
Movie.__doc__

'This class provides a way to store movie related information \n    Second line'

In [50]:
Movie.__name__

'Movie'

In [51]:
Movie.__module__

'__main__'

In [52]:
Movie.__dict__

{'VALID_RATINGS': ['G', 'PG', 'PG-13', 'R'],
 '__doc__': 'This class provides a way to store movie related information \n    Second line',
 '__init__': <function __main__.__init__>,
 '__module__': '__main__',
 'show_trailer': <function __main__.show_trailer>}

In [53]:
Movie.__bases__

()

## Inheritance

In [53]:
class Parent():

    def __init__(self, last_name, eye_color):
        print "Parent constuctor called"
        self.last_name = last_name
        self.eye_color = eye_color
        
    VALID_NAMES = 3
    
    @staticmethod 
    def return_3():
        print Parent.VALID_NAMES
        
    def show_info(self):
        print "Last Name: " + self.last_name
        print "Eye color: " + self.eye_color
        print "Valid name: " + str(Parent.VALID_NAMES)

# class child will inherit or re-use anything that is publicly available in class Parent
# In the following we use inheritance to re-use our code. 
class Child(Parent):
    
    
    def __init__(self, last_name, eye_color, number_of_toys):
        print "Child constructor called"
        Parent.__init__(self, last_name, eye_color)
        self.number_of_toys = number_of_toys
    
    def show_info(self):
        print "Last Name: " + self.last_name
        print "Eye Color: " + self.eye_color
        print "number of toys: " + str(self.number_of_toys)

In [56]:
p = Parent("Cyrus", "Blue")

Parent constuctor called


In [54]:
Parent.return_3()

3


In [17]:
c = Child("Cyrus", "Blue", 5)

Child constructor called
Parent constuctor called


In [57]:
p.show_info()

Last Name: Cyrus
Eye color: Blue
Valid name: 3


In [31]:
Child.return_3()

3


In [3]:
billy_cyrus = Parent("Cyruss", "blue")

Parent constuctor called


In [4]:
billy_cyrus.eye_color

'blue'

In [86]:
miley_cyrus = Child("Cyrus", "Blue", 5)

Child constructor called
Parent constuctor called


In [64]:
miley_cyrus.eye_color

'Blue'

In [70]:
miley_cyrus.last_name

'Cyrus'

In [71]:
billy_cyrus.last_name

'Cyruss'

In [75]:
billy_cyrus.show_info()

Last Name: Cyruss
Eye color: blue


Since Child is an inherited class, instances of class Child such as miley_cyrus can also call the show_info() method even though it is not explicitly defined inside the class Child

In [77]:
miley_cyrus.show_info()

Last Name: Cyrus
Eye color: Blue


When we define a method with the same name in the subclass and use it on the instance of the subclass, then the definition inside the subclass is used. This is called **Method overriding**

In [87]:
miley_cyrus.show_info()

Last Name: Cyrus
Eye Color: Blue
number of toys: 5


## Datacamp tutorial

In [1]:
class Dog:
    
    def __init__(self):
        
        pass

In [2]:
axle = Dog()

In [3]:
print(axle)

**Adding attributes to the class**

In [4]:
class Dog:
    
    def __init__(self, name, age):
        
        self.name = name
        self.age = age

In [9]:
axle = Dog("Axle", 3)

The attributes are accessed using the dot method

In [10]:
axle.name

'Axle'

In [11]:
axle.age

3

**Adding methods to the class**

In [1]:
class Dog:
    
    def __init__(self, name, age):
        
        self.name = name
        self.age = age
        
    def bark(self):
        
        print("Bhow bhow!")

In [2]:
axle = Dog("Axle", 3)

In [3]:
axle.name

'Axle'

In [4]:
axle.bark()

Bhow bhow!
